In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Print the current working directory, help ensure you're able to load dataset from correct path
print("Current Working Directory:", os.getcwd())

# Load the dataset from your csv file
df = pd.read_csv("./StockMarketDataset.csv")

In [ ]:
# Lists all the columns
print(df.columns.tolist())

# Dropping first column as it just mirrors the row number, no meaningful data
df.drop(columns=['Unnamed: 0'], inplace=True)

# Lists all the columns
print(f'Columns after dropping the first column: {df.columns.tolist()}')

In [ ]:
# Checks for missing (null/NaN) values in a DataFrame (df), returns count of null values in each column
# Example: 

#Returns the count of null values in each column
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
# Parse the date
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, format="%d-%m-%Y")
df = df.sort_values('Date')  # Ensure chronological order
df.reset_index(drop=True, inplace=True)

# List all columns ending in '_Price'
#print('Printing all columns that have _Price in its name')
price_columns = [col for col in df.columns if col.endswith('_Price')]
#print(price_columns)

# Convert them to numeric (in case they are strings)
for col in price_columns:
    # Ensure all values are strings (safe to apply string ops)
    df[col] = df[col].astype(str)
    
    # Remove commas and convert to numeric (force errors to NaN)
    df[col] = df[col].str.replace(',', '', regex=False)

    #print(f'Before conversion: {df[col]}')
    print(f'After conversion: {pd.to_numeric(df[col], errors='coerce')}')

#print('After converting price columns to numeric')
#print(df.head())

# Now drop or fill any NaNs caused by bad entries
df = df.dropna(subset=price_columns)

# Inspecting data, first few rows, last few rows, if any nulls are still present
print(f'First few rows: {df.head()}')
print(f'Last few rows: {df.tail()}')
print(f'Any nulls still present? {df.isna().sum()}')



In [ ]:
# Convert columns to numeric where possible, coercing errors to NaN
df = df.apply(pd.to_numeric, errors='coerce')

# 1. Compute percent missing per column
missing_pct = df.isna().mean()

# 2. Define threshold (e.g., 5%)
threshold = 0.05

# 3. Identify low- and high-missing columns
cols_impute = missing_pct[missing_pct < threshold].index.tolist()
cols_drop = missing_pct[missing_pct >= threshold].index.tolist()
print(f'Columns to drop (≥{threshold*100}% missing): {cols_drop}')

# 4. Impute median for those columns
medians = {col: df[col].median() for col in cols_impute}
df[cols_impute] = df[cols_impute].fillna(medians)

# 5. Drop high-missing columns
df = df.drop(columns=cols_drop)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(df)

In [ ]:
print(f'Any nulls still present? {df.isna().sum()}')

In [ ]:
# Get all columns that represent asset prices (end in '_Price')
price_columns = [col for col in df.columns if col.endswith('_Price')]

# Calculate daily percentage returns
returns = df[price_columns].pct_change()

# Rolling volatility calculations
volatility_7 = returns.rolling(window=7).std()
volatility_14 = returns.rolling(window=14).std()
volatility_21 = returns.rolling(window=21).std()

# Rename columns
volatility_7.columns = [f'{col}_vol_7d' for col in volatility_7.columns]
volatility_14.columns = [f'{col}_vol_14d' for col in volatility_14.columns]
volatility_21.columns = [f'{col}_vol_21d' for col in volatility_21.columns]





In [ ]:
# Merge back to the original df
df = pd.concat([df, volatility_7, volatility_14, volatility_21], axis=1)
df = df.dropna()  # Drop rows with NaNs introduced by rolling windows

In [ ]:
#df[['Date', 'Bitcoin_Price', 'Bitcoin_Price_vol_7d', 'Bitcoin_Price_vol_14d', 'Bitcoin_Price_vol_21d']].head()
print(df)

In [ ]:
# Reset index example
print('''Code example:\n pd.DataFrame({ 
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35]
    }, index=['a', 'b', 'c'])''')

df = pd.DataFrame({ 
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35]
    }, index=['a', 'b', 'c'])

print('\nNow we remove \'Bob\' with:\n df = df[df[\'Age\'] != 30]') 
df = df[df['Age'] != 30]

print(f'If we don\'t reset index, we get:\n {df}')
df.reset_index(drop=True, inplace=True)
print(f'After we reset index, we get:\n {df}')

In [2]:
## 1. Load csv file
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# For debugging, help ensure you're able to load dataset from correct path
print("Current Working Directory:", os.getcwd())

# Load the dataset from your csv file
df = pd.read_csv("./StockMarketDataset.csv")

## 2. Sanitize dataset
# The first column is used for the row number, unnecessary, dropping it
df.drop(columns=['Unnamed: 0'], inplace=True)

# Parse and sort dates in chronological order
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, format="%d-%m-%Y")
df = df.sort_values('Date')  # Ensure chronological order
df.reset_index(drop=True, inplace=True)

# Removing commas from price columns and convert to numeric
price_columns = [col for col in df.columns if col.endswith('_Price')]

for col in price_columns:
    # Ensure all values are strings (safe to apply string ops)
    df[col] = df[col].astype(str)
    
    # Remove commas and convert to numeric (force errors to NaN)
    df[col] = df[col].str.replace(',', '', regex=False)

df = df.apply(pd.to_numeric, errors='coerce')

## 3. Impute missing data with mean and drop
# Compute percent missing per column
missing_pct = df.isna().mean()

# Define threshold (e.g., 5%)
threshold = 0.05

# Identify low-missing and high-missing columns
cols_impute = missing_pct[missing_pct < threshold].index.tolist()
cols_drop = missing_pct[missing_pct >= threshold].index.tolist()

# Impute median on low missing columns
medians = {col: df[col].median() for col in cols_impute}
df[cols_impute] = df[cols_impute].fillna(medians)

# Drop high-missing columns
df = df.drop(columns=cols_drop)

## 4. Calculate rolling volatility
# Calculate daily percentage returns
returns = df[price_columns].pct_change()

# Rolling volatility calculations
volatility_7 = returns.rolling(window=7).std()
volatility_14 = returns.rolling(window=14).std()
volatility_21 = returns.rolling(window=21).std()

# Rename columns
volatility_7.columns = [f'{col}_vol_7d' for col in volatility_7.columns]
volatility_14.columns = [f'{col}_vol_14d' for col in volatility_14.columns]
volatility_21.columns = [f'{col}_vol_21d' for col in volatility_21.columns]

df = pd.concat([df, volatility_7, volatility_14, volatility_21], axis=1)
df = df.dropna()  # Drop rows with NaNs introduced by rolling windows

## Save cleaned up dataset!
print("Cleaned dataset:")
print(df)
df.to_csv('StockMarketDataset_Sanitized.csv', index=False)


Current Working Directory: /Users/gjbaptiste/Dev/Repos/Playground/Python
Cleaned dataset:
                     Date  Natural_Gas_Price  Natural_Gas_Vol.  \
21    1551830400000000000              2.841          119630.0   
22    1551916800000000000              2.866          117000.0   
23    1552003200000000000              2.865           85880.0   
24    1552262400000000000              2.772          136890.0   
25    1552348800000000000              2.784           90020.0   
...                   ...                ...               ...   
1238  1706486400000000000              2.490            3590.0   
1239  1706572800000000000              2.077          139750.0   
1240  1706659200000000000              2.100          142860.0   
1241  1706745600000000000              2.050          161340.0   
1242  1706832000000000000              2.079          127370.0   

      Crude_oil_Price  Crude_oil_Vol.  Copper_Price  Copper_Vol.  \
21              56.22        637680.0        2.91

In [2]:
# Finalized Version, Sanitized and Labeled for Machine Learning training

import pandas as pd
import numpy as np
import os

# 1. Load CSV file
print("Current Directory:", os.getcwd())
df = pd.read_csv("./StockMarketDataset.csv")

# 2. Sanitize dataset
df.drop(columns=['Unnamed: 0'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, format="%d-%m-%Y")
df = df.sort_values('Date').reset_index(drop=True)

price_cols = [c for c in df.columns if c.endswith('_Price')]
for col in price_cols:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False)
df = df.apply(pd.to_numeric, errors='coerce')

# 3. Impute missing & drop high-missing
missing_pct = df.isna().mean()
threshold = 0.05
cols_impute = [c for c in df if missing_pct[c] < threshold]
cols_drop = [c for c in df if missing_pct[c] >= threshold]
df[cols_impute] = df[cols_impute].fillna(df[cols_impute].median())
df.drop(columns=cols_drop, inplace=True)

# 4. Compute rolling volatilities
returns = df[price_cols].pct_change()
vol_7 = returns.rolling(7).std().add_suffix('_vol_7d')
vol_14 = returns.rolling(14).std().add_suffix('_vol_14d')
vol_21 = returns.rolling(21).std().add_suffix('_vol_21d')

df = pd.concat([df, vol_7, vol_14, vol_21], axis=1).dropna().reset_index(drop=True)

# 5. Quantile-based labeling per asset & window
for window in ['7d', '14d', '21d']:
    vol_cols = [c for c in df.columns if c.endswith(f'_vol_{window}')]
    for col in vol_cols:
        q_low, q_high = df[col].quantile([0.33, 0.66])
        label_col = f'{col}_Risk'
        df[label_col] = df[col].apply(
            lambda v: 'Low' if v <= q_low else 'Medium' if v <= q_high else 'High'
        )
        print(f"{label_col}: ≤{q_low:.4f} Low; ≤{q_high:.4f} Med; else High")

# 6. One-hot encode all Risk label columns
risk_cols = [c for c in df.columns if c.endswith('_Risk')]
df = pd.concat([df, pd.get_dummies(df[risk_cols], prefix=risk_cols)], axis=1)

# 7. Save final dataset with labels
print("Sample of Risk labels:")
print(df[risk_cols].head())
print(df[risk_cols].tail())
df.to_csv('StockMarketDataset_Sanitized_Labeled.csv', index=False)


Current Directory: /Users/gjbaptiste/Dev/Repos/Playground/Python
Natural_Gas_Price_vol_7d_Risk: ≤0.0268 Low; ≤0.0445 Med; else High
Crude_oil_Price_vol_7d_Risk: ≤0.0167 Low; ≤0.0252 Med; else High
Copper_Price_vol_7d_Risk: ≤0.0102 Low; ≤0.0144 Med; else High
Bitcoin_Price_vol_7d_Risk: ≤0.0259 Low; ≤0.0409 Med; else High
Platinum_Price_vol_7d_Risk: ≤0.0144 Low; ≤0.0200 Med; else High
Ethereum_Price_vol_7d_Risk: ≤0.0339 Low; ≤0.0504 Med; else High
S&P_500_Price_vol_7d_Risk: ≤0.0067 Low; ≤0.0110 Med; else High
Nasdaq_100_Price_vol_7d_Risk: ≤0.0093 Low; ≤0.0154 Med; else High
Apple_Price_vol_7d_Risk: ≤0.0125 Low; ≤0.0190 Med; else High
Tesla_Price_vol_7d_Risk: ≤0.0259 Low; ≤0.0398 Med; else High
Microsoft_Price_vol_7d_Risk: ≤0.0117 Low; ≤0.0181 Med; else High
Silver_Price_vol_7d_Risk: ≤0.0130 Low; ≤0.0194 Med; else High
Google_Price_vol_7d_Risk: ≤0.0124 Low; ≤0.0191 Med; else High
Nvidia_Price_vol_7d_Risk: ≤0.0215 Low; ≤0.0315 Med; else High
Berkshire_Price_vol_7d_Risk: ≤0.0079 Low; ≤0.011